In [65]:
%reload_ext autoreload

%autoreload 2
import numpy as np
from models import lstm
import torch
from sklearn.preprocessing import StandardScaler
import random
import pickle

In [66]:
# FILES TO CHANGE (This is where your sequence_nlp_harvey.zip etc is) test

NPY_INPUT_DIR = '/Users/ianmagnusson/IITUDND/data/extracted_features/combined_NLP/harvey/kfold/'

OUT_FILE = '/Users/ianmagnusson/IITUDND/notebooks/harvey_random_LSTM_results.npy'

NUM_EXPERIMENTS = 10000000000000000

In [67]:
def run_experiment(input_dim, hidden_dim, num_layers, bidirectional,  learning_rate, momentum, decay_factor, epochs, data_dir):
    
    # load data from files
    
    X_seq_tfidf_train = np.load(data_dir + 'X_seq_tfidf_train.npz')
    X_seq_tfidf_test = np.load(data_dir + 'X_seq_tfidf_test.npz')
    
    
    X_seq_glove_train = np.load(data_dir + 'X_seq_glove_train.npz')
    X_seq_glove_test = np.load(data_dir + 'X_seq_glove_test.npz')
    
    y_train = torch.Tensor(np.load(data_dir + 'y_train.npy'))
    y_test = torch.Tensor(np.load(data_dir + 'y_test.npy'))
    
    # build lists with features concatenated
    
    X_raw_train = []
    for i in range(len(y_train)):
        file = X_seq_tfidf_train.files[i]
        X_raw_train.append(np.concatenate((X_seq_tfidf_train[file], X_seq_glove_train[file]), axis=1))
    
    scaler = StandardScaler()
    X_fit_train = np.concatenate(X_raw_train)
    scaler.fit(X_fit_train)
    
    X_train = [torch.Tensor(scaler.transform(X_i)) for X_i in X_raw_train]
    
    X_test = []
    for i in range(len(y_test)):
        file = X_seq_tfidf_test.files[i]
        X_test.append(torch.Tensor(scaler.transform(np.concatenate((X_seq_tfidf_test[file], X_seq_glove_test[file]), axis=1))))

    
    # build and train model
    model = lstm.LSTM(input_dim=input_dim, hidden_dim=hidden_dim, num_layers = num_layers, bidirectional = bidirectional,
                      learning_rate=learning_rate, momentum = momentum, decay_factor = decay_factor)
    
    model.learn(X_train, y_train, epochs = epochs)
    
    # evaluate
    return model.get_accuracy(X_test, y_test)

def cross_validate(input_dim, hidden_dim, num_layers, bidirectional,  learning_rate, momentum, decay_factor, epochs,
                   data_dir, folds = 10):
    accuracy_sum = 0.0
    x = 0
    for i in range(folds):
        fold_dir = data_dir + str(i) + '/'
        print('running fold', i)
        acc = run_experiment(input_dim, hidden_dim, num_layers, bidirectional,
                             learning_rate, momentum, decay_factor,epochs,fold_dir)
        print('fold result', acc)
        accuracy_sum += acc
        
    return accuracy_sum / folds


In [68]:
def get_random_params(range_hd, range_nl, range_lr, range_mo, range_dr, range_ep):
    hidden_dim = random.randint(*range_hd)
    num_layers = random.randint(*range_nl)
    bidirectional = bool(random.getrandbits(1))
    learning_rate = random.uniform(*range_lr)
    momentum = random.uniform(*range_mo)
    decay_factor = random.uniform(*range_dr)
    epochs = random.randint(*range_ep)
    
    return hidden_dim, num_layers, bidirectional,  learning_rate, momentum, decay_factor, epochs

In [ ]:
range_hd = (50,1500)
range_nl = (1, 4)
range_lr = (.1, .0001)
range_mo = (0,1)
range_dr = (.1,1)
range_ep = (1,10)

# run this instead for first time
#results = []

# cross validate search for hyper-parameters
with open(OUT_FILE, 'rb') as fin:
    results = pickle.load(fin)
    
for i in range(NUM_EXPERIMENTS):
    hyperparams = get_random_params(range_hd, range_nl, range_lr, range_mo, range_dr, range_ep)
    print('starting test', i,'params:', hyperparams)
    mean_acc = cross_validate(400, *hyperparams, NPY_INPUT_DIR)
    print('test outcome', mean_acc)
    print('************************************************************')
    results.append((mean_acc, hyperparams))
    results = sorted(results, key=lambda x: x[0], reverse=True)
    print('top so far')
    print(results[:1])
    print('************************************************************')
    
    # backup results
    with open(OUT_FILE, 'wb') as fout:
        pickle.dump(results, fout)



starting test 0 params: (1315, 4, True, 0.05126334703809319, 0.0066123569921588965, 0.5964792048201312, 1)
running fold 0
epoch: 0 learning rate: [0.05126334703809319]


In [ ]:
results = sorted(results, key=lambda x: x[0], reverse=True)

print(results[:3])


# RUN THIS CELL AFTER YOU HAVE INTERUPTED OR FINISHED A SEARCH TO UPDATE!!!!!!

with open(OUT_FILE, 'wb') as fout:
    pickle.dump(results, fout)
